# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [16]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import pandas as pd
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 5b73622f-88cd-4735-ad7f-55b8fad69d90.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 5b73622f-88cd-4735-ad7f-55b8fad69d90.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 5b73622f-88cd-4735-ad7f-55b8fad69d90.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 5b73622f-88cd-4735-ad7f-55b8fad69d90.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [2]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='sales', table_name='balancesheet10')
dyf.printSchema()

root
|-- date: string
|-- symbol: string
|-- reportedCurrency: string
|-- cik: string
|-- fillingDate: string
|-- acceptedDate: string
|-- calendarYear: string
|-- period: string
|-- cashAndCashEquivalents: choice
|    |-- int
|    |-- long
|-- shortTermInvestments: choice
|    |-- int
|    |-- long
|-- cashAndShortTermInvestments: long
|-- netReceivables: choice
|    |-- int
|    |-- long
|-- inventory: choice
|    |-- int
|    |-- long
|-- otherCurrentAssets: choice
|    |-- int
|    |-- long
|-- totalCurrentAssets: long
|-- propertyPlantEquipmentNet: choice
|    |-- int
|    |-- long
|-- goodwill: choice
|    |-- int
|    |-- long
|-- intangibleAssets: choice
|    |-- int
|    |-- long
|-- goodwillAndIntangibleAssets: choice
|    |-- int
|    |-- long
|-- longTermInvestments: choice
|    |-- int
|    |-- long
|-- taxAssets: choice
|    |-- int
|    |-- long
|-- otherNonCurrentAssets: choice
|    |-- int
|    |-- long
|-- totalNonCurrentAssets: long
|-- otherAssets: int
|-- totalAsse

#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [4]:
df = dyf.toDF()
df.show(1)

+----------+------+----------------+----------+-----------+-------------------+------------+------+----------------------+--------------------+---------------------------+-------------------+------------------+-------------------+------------------+-------------------------+---------+----------------+---------------------------+--------------------+---------+---------------------+---------------------+-----------+------------+-------------------+-------------------+-----------+------------------+-----------------------+-----------------------+-------------------+-------------------------+--------------------------------+--------------------------+--------------------------+----------------+-----------------------+----------------+--------------+-------------------+-------------------+---------------------------------------+----------------------------+-----------------------+-----------+-------------------------------------+----------------+------------------------------+--------------

In [21]:
df2 = DropNullFields.apply(dyf)
print(type(df2))

null_fields []
<class 'awsglue.dynamicframe.DynamicFrame'>


In [22]:
df2.show(1)

{"date": "2022-09-24", "symbol": "AAPL", "reportedCurrency": "USD", "cik": "0000320193", "fillingDate": "2022-10-28", "acceptedDate": "2022-10-27 18:01:14", "calendarYear": "2022", "period": "FY", "cashAndCashEquivalents": 23646000000, "shortTermInvestments": 24658000000, "cashAndShortTermInvestments": 48304000000, "netReceivables": 60932000000, "inventory": 4946000000, "otherCurrentAssets": 21223000000, "totalCurrentAssets": 135405000000, "propertyPlantEquipmentNet": 42117000000, "goodwill": 0, "intangibleAssets": 0, "goodwillAndIntangibleAssets": 0, "longTermInvestments": 120805000000, "taxAssets": 0, "otherNonCurrentAssets": 54428000000, "totalNonCurrentAssets": 217350000000, "otherAssets": 0, "totalAssets": 352755000000, "accountPayables": 64115000000, "shortTermDebt": 21110000000, "taxPayables": 0, "deferredRevenue": 7912000000, "otherCurrentLiabilities": 60845000000, "totalCurrentLiabilities": 153982000000, "longTermDebt": 98959000000, "deferredRevenueNonCurrent": 0, "deferredTax

In [60]:
glueContext.write_dynamic_frame.from_options(frame = df2, connection_type = "s3", connection_options = {"path": "s3:"},format="json",transformation_ctx = "DataSink0")
job.commit()

In [61]:
glueContext.write_dynamic_frame.from_options(
    frame=df2,
    connection_type="s3",
    connection_options={"path": "s3:"},
    format="json",
    transformation_ctx="DataSink0"
)

job.commit()

In [ ]:
my_conn = {
    "url" : "jdbc:redshift://.redshift-serverless.amazonaws.com:5439/dev",
    "user": "admin",
    "password": "",
    "dbtable" : "]",
    "database" : ""
}


redshift_data = glueContext.write_dynamic_frame.from_jdbc_conf(df2,
    catalog_connection = "redshiftconnection",
    connection_options = my_conn,
    redshift_tmp_dir = "s3://")